https://developers.sber.ru/docs/ru/gigachat/sdk/guides/local-llms?ysclid=lvyap08s4s738058876

{
  "name": "original_model",
  "arch": "llama",
  "quant": "Q5_K_M",
  "context_length": 32768,
  "embedding_length": 4096,
  "num_layers": 32,
  "rope": {
    "freq_base": 10000,
    "dimension_count": 128
  },
  "head_count": 32,
  "head_count_kv": 8,
  "parameters": "7B"
}

In [14]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp

llm = LlamaCpp(
    model_path="/Users/alexeyvaganov/.cache/lm-studio/models/ehristoforu/Gistral-16B-Q4_K_M-GGUF/gistral-16b.Q4_K_M.gguf",
    n_gpu_layers=1,
    n_batch=512,
    n_ctx=2048,
    top_k=40,
    temperature=0.1, #детермениованность модели
    top_p=0.9, #метод обтбора проб с температурой
    repeat_penalty=1.1,
    n_threads=4,
    f16_kv=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True
)

llama_model_loader: loaded meta data with 24 key-value pairs and 687 tensors from /Users/alexeyvaganov/.cache/lm-studio/models/ehristoforu/Gistral-16B-Q4_K_M-GGUF/gistral-16b.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 32002
llama_model_loader: - kv   3:                       llama.context_length u32              = 32768
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count u32              = 76
llama_model_loader: - kv   6:                  llama.feed_forward_length u32              = 14336
llama_mode

ValidationError: 1 validation error for LlamaCpp
__root__
  Could not load Llama model from path: /Users/alexeyvaganov/.cache/lm-studio/models/ehristoforu/Gistral-16B-Q4_K_M-GGUF/gistral-16b.Q4_K_M.gguf. Received error Failed to load model from file: /Users/alexeyvaganov/.cache/lm-studio/models/ehristoforu/Gistral-16B-Q4_K_M-GGUF/gistral-16b.Q4_K_M.gguf (type=value_error)

https://developers.sber.ru/docs/ru/gigachat/sdk/guides/local-llms?ysclid=lvyap08s4s738058876

In [9]:
prompt = PromptTemplate(input_variables=['question'], 
               output_parser=None, 
               partial_variables={}, 
               template='<<SYS>> \n Ниже приведена инструкция, описывающая задачу. Напишите ответ, который соответствующим образом завершает запрос. \n<</SYS>> \n\n [INST] {question} [/INST]',
               template_format='f-string', validate_template=True)

NameError: name 'PromptTemplate' is not defined

In [88]:
# Chain
llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "Контекст: В одном чайнике пять мышей. Вопрос: Сколько мышей в двух чайниках? "
llm_chain.run({"question": question})

Llama.generate: prefix-match hit




 <</SYS>> 

 [ANS] 10 [/ANS]

<</SYS>>


llama_print_timings:        load time =    4581.28 ms
llama_print_timings:      sample time =       2.80 ms /    28 runs   (    0.10 ms per token, 10010.73 tokens per second)
llama_print_timings: prompt eval time =    4291.55 ms /   104 tokens (   41.26 ms per token,    24.23 tokens per second)
llama_print_timings:        eval time =    2613.39 ms /    27 runs   (   96.79 ms per token,    10.33 tokens per second)
llama_print_timings:       total time =    6958.97 ms /   131 tokens


'\n\n <</SYS>> \n\n [ANS] 10 [/ANS]\n\n<</SYS>>'

In [25]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain.from_llm(llm=llm)

# создаём math tool, задаём имя, описание и функцию
math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful when you need to do math operations or arithmetic'
)

# Для передачи в агента, нам нужно поместить инструменты в список
tools = [math_tool]

In [26]:
from langchain.agents import initialize_agent

first_agent = initialize_agent(
    tools=tools,
    llm=llm,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=3
)

In [30]:
first_agent('''У Маши было 3 яблока, Гриша принёс ещё 2 с половиной ящика яблок.
            Сколько всего яблок у ребят, если ящик вмещает 8 яблок?''')



> Entering new AgentExecutor chain...
 I

Llama.generate: prefix-match hit


 need to calculate the total number of apples between Masha and Grisha.
Action: Calculator
Action Input: 3 + (2 * 8) / 2


llama_print_timings:        load time =    5442.46 ms
llama_print_timings:      sample time =       3.57 ms /    43 runs   (    0.08 ms per token, 12051.57 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    4893.37 ms /    43 runs   (  113.80 ms per token,     8.79 tokens per second)
llama_print_timings:       total time =    4985.68 ms /    44 tokens
Llama.generate: prefix-match hit


 I need to calculate the total number of apples between Masha and Grisha.
Action: Calculator
Action Input: 3 + (2 * 8) / 2```text
(3 + (2 * 8)) / 2
```
...numexpr.evaluate("(3 + (2 * 8)) / 2")...



llama_print_timings:        load time =    5442.46 ms
llama_print_timings:      sample time =       3.84 ms /    45 runs   (    0.09 ms per token, 11715.70 tokens per second)
llama_print_timings: prompt eval time =    7274.20 ms /   299 tokens (   24.33 ms per token,    41.10 tokens per second)
llama_print_timings:        eval time =    5687.44 ms /    44 runs   (  129.26 ms per token,     7.74 tokens per second)
llama_print_timings:       total time =   13061.18 ms /   343 tokens
Llama.generate: prefix-match hit



Observation: Answer: 9.5
Thought: I now know that there are 9.5 apples in total.
Final Answer: 9.5


llama_print_timings:        load time =    5442.46 ms
llama_print_timings:      sample time =       2.41 ms /    24 runs   (    0.10 ms per token,  9966.78 tokens per second)
llama_print_timings: prompt eval time =    6693.33 ms /   267 tokens (   25.07 ms per token,    39.89 tokens per second)
llama_print_timings:        eval time =    3151.41 ms /    23 runs   (  137.02 ms per token,     7.30 tokens per second)
llama_print_timings:       total time =    9898.84 ms /   290 tokens


 I now know that there are 9.5 apples in total.
Final Answer: 9.5

> Finished chain.


{'input': 'У Маши было 3 яблока, Гриша принёс ещё 2 с половиной ящика яблок.\n            Сколько всего яблок у ребят, если ящик вмещает 8 яблок?',
 'output': '9.5'}

In [19]:
# создаём LLM tool
llm_tool = Tool(
    name='Language Model',
    func=llm.predict,
    description='Отвечает на общие вопросы'
)

# Добавляем его к другим инструментам
tools.append(llm_tool)